# ELMo

Note that you will need to use the non-GPU accelerated run-time on this notebook due to the large memory useage of the ELMo model.

## Imports:

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing

os.environ['TFHUB_CACHE_DIR'] = 'D:/danac/Documents/Info/M&K/m-k-manuscript-data/manuscript-object/tf_cache'

If the below cell does not work on first try, restart the kernel and try again

In [2]:
#!python -m spacy download en_core_web_md
import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en_core_web_md')

In [3]:
from IPython.display import HTML
import logging
#logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

## Get the data 

The below downloads a Pandas Dataframe which is publically hosted on Google Drive (this should therefore work for anyone)

(this is from an earlier version and serves no purpose but I'm keeping it because it looks cool)

In [4]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


file_id = '1M_XljfV5t_nGjvhyfTPO9n2nfOweMwYx'
destination = 'temp'
download_file_from_google_drive(file_id, destination)

combined = pd.read_pickle('temp')

combined.head()

,Company,URL,Industry,HQ,Also Covers Companies,UK Modern Slavery Act,California Transparency in Supply Chains Act,Period Covered,text,pdf,error,FT_tfidf
0,118 118 Money,https://www.118118money.com/anti-slavery-state...,Consumer Finance,United Kingdom,NaN,True,False,2016-2017,Introduction\n\nThis statement is made pursuan...,0,0,"[-0.09019677307999371, 0.23825215930123844, 0...."
1,1Spatial Plc,https://1spatial.com/who-we-are/legal/modern-s...,Software,United Kingdom,NaN,True,False,2017,While 1Spatial’s turnover is below £36m and th...,0,0,"[-0.5010607985753625, 0.42660413175930045, -0...."
2,20/20 Projects,http://www.2020projects.co.uk/modernslaverypolicy,Commercial Services & Supplies,United Kingdom,NaN,True,False,2015-2016,html error,0,1,"[0.9405487179756165, 0.40332984924316406, 0.70..."
3,2M Holdings Ltd,https://www.2m-holdings.com/2m-holdings-ltd-mo...,Distributors,United Kingdom,NaN,True,False,2015-2016,Modern slavery is a crime resulting in an abho...,0,0,"[-0.390252637821283, 0.488747594191321, -0.238..."
4,3i Group plc,https://www.3i.com/media/3815/modern-slavery-s...,Capital Markets,United Kingdom,NaN,True,False,2017-2018,pdf error tika,1,1,"[1.0879868624172204, 0.44540999591903685, 0.80..."


## Create sentence embeddings

In [4]:
url = "https://tfhub.dev/google/elmo/3"
path2="D:/danac/Documents/Info/M&K/m-k-manuscript-data/manuscript-object/ELMo2"
path3="D:/danac/Documents/Info/M&K/m-k-manuscript-data/manuscript-object/ELMo3"  #this is ELMo3
embed = hub.Module(path3)

In [10]:
combined.loc[combined.Company.str.contains("Asos")]

,Company,URL,Industry,HQ,Also Covers Companies,UK Modern Slavery Act,California Transparency in Supply Chains Act,Period Covered,text,pdf,error,FT_tfidf
494,Asos plc,https://www.asosplc.com/~/media/Files/A/Asos-V...,Internet & Direct Marketing Retail,United Kingdom,NaN,True,False,2016-2018,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1,0,"[-0.8573993510860287, 0.11926992131730585, -0...."
495,Asos plc,https://www.asosplc.com/~/media/Files/A/ASOS/r...,Internet & Direct Marketing Retail,United Kingdom,NaN,True,False,2015-2016,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1,0,"[-0.6513810935113411, 0.04600498146602333, 0.0..."


In [10]:
#text = combined.iloc[494].text
import re


fileo = open('D:/danac/Documents/Info/M&K/m-k-manuscript-data/allFolios/txt/all_tl.txt','r',encoding='utf-8')
text=fileo.read()
fileo.close()

text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')
text = ' '.join(text.split())
doc = nlp(text)

counter=0
sentences = []
for i in doc.sents:
  if len(i)>1 and counter<400:
    sentences.append(i.string.strip())
    counter+=1
    
len(sentences)

400

In [11]:
sentences[100:110]

['and as you see that the old varnish is removed, throw a bucket of water against the panel to clean it.',
 'next put it for a quarter of an hour in the sun to make it dry and to revive the colors.',
 'next, once the panel is dry, you can spread your turpentine varnish on it.',
 'make sure the colors do not come off.',
 'black varnish for sword guard, bands for trunks, &amp;c take linseed oil or more cheaply, walnut oil, and rid it of grease with garlic &amp; onions +hog’s fennel',
 ', some also add bread crusts, which you will boil in it for a good quarter of an hour.',
 'next, put in one lb of the oil thus boiled the size of a walnut of black pitch &amp; a double handful of grains of wheat, without removing the garlic &amp; onions, and let it boil together for a good quarter of an hour.',
 'and when the pitch is well melted &amp; when the oil has body, you can remove it from the fire.',
 'then, to varnish, place your iron over a low charcoal fire &amp; apply your varnish with a feath

In [12]:
embeddings = embed(
    sentences,
    signature="default",
    as_dict=True)["default"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
%%time
gpu_options = tf.GPUOptions(allow_growth=True) 
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  x = sess.run(embeddings)

ResourceExhaustedError: OOM when allocating tensor with shape[5608,151,46,256] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node module_apply_default/bilm/CNN/Conv2D_4 (defined at D:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'module_apply_default/bilm/CNN/Conv2D_4':
  File "D:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File

## Visualize the sentences using PCA and TSNE

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
y = pca.fit_transform(x)

from sklearn.manifold import TSNE

y = TSNE(n_components=2).fit_transform(y)

In [ ]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


data = [
    go.Scatter(
        x=[i[0] for i in y],
        y=[i[1] for i in y],
        mode='markers',
        text=[i for i in sentences],
    marker=dict(
        size=16,
        color = [len(i) for i in sentences], #set color equal to a variable
        opacity= 0.8,
        colorscale='Viridis',
        showscale=False
    )
    )
]
layout = go.Layout()
layout = dict(
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )
fig = go.Figure(data=data, layout=layout)
file = plot(fig, filename='Sentence_encode_elmo3_400.html')

## Create a semantic search engine:

In [25]:
#@title Sementic search
#@markdown Enter a set of words to find matching sentences. 'results_returned' can beused to modify the number of matching sentences retured. To view the code behind this cell, use the menu in the top right to unhide...
search_string = "tourmentine" #@param {type:"string"}
results_returned = "5" #@param [1, 2, 3]

from sklearn.metrics.pairwise import cosine_similarity


embeddings2 = embed(
    [search_string],
    signature="default",
    as_dict=True)["default"]

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  search_vect = sess.run(embeddings2)
  

cosine_similarities = pd.Series(cosine_similarity(search_vect, x).flatten())
output =""
for i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
  output +='<p style="font-family:verdana; font-size:110%;"> '
  for i in sentences[i].split():
    if i.lower() in search_string:
      output += " <b>"+str(i)+"</b>"
    else:
      output += " "+str(i)
  output += "</p><hr>"
    
output = '<h3>Results:</h3>'+output
display(HTML(output))
#   print(sentences[i])
#   print('\n')


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
